In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
nonFraudulentFeatures= pd.read_csv('./nonFraudulentFeatures.csv')
fraudulentFeatures= pd.read_csv('./fraudulentFeatures.csv')
nonFraudulentLable= pd.read_csv('./nonFraudulentLable.csv')
fraudulentLable= pd.read_csv('./fraudulentLable.csv')


In [3]:
#nonFraudulent.head()
fraudulentLable.head()

,lable
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [4]:
#Note that for the train and test split, the lables and data have to be in numpy
nonFraudulentCarfeatures= nonFraudulentFeatures.values
nonFraudulentLable= nonFraudulentLable.values

fraudulentCarfeatures= fraudulentFeatures.values
fraudulentLable= fraudulentLable.values

print('non-Fraud: %d' %(nonFraudulentCarfeatures.shape[0]))
print('Fraud: %d' %(fraudulentCarfeatures.shape[0]))


non-Fraud: 14496
Fraud: 923


In [5]:
#divide the nonFraudulent into test and train
nonFraudX_train,nonFraudX_test,nonFraudY_train,nonFraudY_test = train_test_split(nonFraudulentCarfeatures,nonFraudulentLable,random_state=3,test_size=0.25)
print('nonFraudxtrain:',type(nonFraudX_train))
print(nonFraudX_test.shape)

#divide the fraudulent into test and train
fraudX_train,fraudX_test,fraudY_train,fraudY_test = train_test_split(fraudulentCarfeatures, fraudulentLable,random_state=3,test_size=0.25)
print('fraudxtrain:',type(fraudX_train))
print(fraudX_test.shape)


nonFraudxtrain: <class 'numpy.ndarray'>
(3624, 97)
fraudxtrain: <class 'numpy.ndarray'>
(231, 97)


In [6]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [29]:
#make the  nonfraudulent dataset for train and test loader
print(type(nonFraudX_train))
myNonFraudulentTrainDataset= myDataset(nonFraudX_train,nonFraudY_train)
myNonFraudulentTestDataset= myDataset(nonFraudX_test,nonFraudY_test)

#make the fraudulent dataset for train and test loader
myFraudulentTrainDataset= myDataset(fraudX_train,fraudY_train)
myFraudulentTestDataset= myDataset(fraudX_test,fraudY_test)

<class 'numpy.ndarray'>


In [8]:
feature,lable= myNonFraudulentTrainDataset.__getitem__(0)
print("feature:",feature)
print("lable:",lable)
print(feature.size())

feature: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.2500, 0.0000, 0.0641])
lable: tensor([0.])
torch.Size([97])


In [9]:

#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentTrainLoader= torch.utils.data.DataLoader(myNonFraudulentTrainDataset,batch_size=1,shuffle=True,num_workers=0)
nonFraudulentTestLoader= torch.utils.data.DataLoader(myNonFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)

fraudulentTrainLoader= torch.utils.data.DataLoader(myFraudulentTrainDataset,batch_size=1,shuffle=True,num_workers=0)
fraudulentTestLoader= torch.utils.data.DataLoader(myFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)



In [10]:
#Network architecture for the base autoencoders
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder1= nn.Sequential(
            nn.Linear(97,70),
            nn.LeakyReLU(True),
            nn.Linear(70,60),
            nn.LeakyReLU(True),
            nn.Linear(60,45),
            nn.LeakyReLU(True),
            nn.Linear(45,20),nn.LeakyReLU(True),nn.Linear(20,10))
        
        self.decoder1= nn.Sequential(
            nn.Linear(10,20),
            nn.LeakyReLU(True),
            nn.Linear(20,45),
            nn.LeakyReLU(True),
            nn.Linear(45,60),
            nn.LeakyReLU(True),
            nn.Linear(60,70),nn.LeakyReLU(True),nn.Linear(70,97),nn.ReLU())
    
    def forward(self,x):
        x= self.encoder1(x)
        x= self.decoder1(x)
        return x;

In [11]:
model1= autoencoder()
criterion1= nn.MSELoss()
optimizer1= torch.optim.SGD(model1.parameters(), lr=0.001,momentum=0.9)

In [12]:
#train the model
for epoch in range(50):
    running_loss= 0.0
    for i, (feature1,lable1) in enumerate(nonFraudulentTrainLoader):
        
        #gets the inputs
        inputs1= torch.tensor(feature1)
        lables1= torch.tensor(lable1)
        lables1= lables1.type(torch.LongTensor)
        
        # =====================forward====================
        output1 = model1(inputs1)
        loss1 = criterion1(output1,inputs1)
        
         # ===================backward====================
        optimizer1.zero_grad()
        loss1.backward()
        optimizer1.step()
        
        # =======print the statistics
        running_loss += loss1.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss /nonFraudX_train.shape[0]))
      #running_loss = 0.0

        

[1] loss: 0.113
[2] loss: 0.107
[3] loss: 0.107
[4] loss: 0.107
[5] loss: 0.107
[6] loss: 0.106
[7] loss: 0.104
[8] loss: 0.090
[9] loss: 0.080
[10] loss: 0.078
[11] loss: 0.076
[12] loss: 0.076
[13] loss: 0.068
[14] loss: 0.065
[15] loss: 0.064
[16] loss: 0.063
[17] loss: 0.061
[18] loss: 0.058
[19] loss: 0.057
[20] loss: 0.057
[21] loss: 0.055
[22] loss: 0.053
[23] loss: 0.052
[24] loss: 0.052
[25] loss: 0.052
[26] loss: 0.052
[27] loss: 0.051
[28] loss: 0.051
[29] loss: 0.051
[30] loss: 0.051
[31] loss: 0.051
[32] loss: 0.051
[33] loss: 0.051
[34] loss: 0.051
[35] loss: 0.051
[36] loss: 0.051
[37] loss: 0.051
[38] loss: 0.050
[39] loss: 0.049
[40] loss: 0.048
[41] loss: 0.047
[42] loss: 0.047
[43] loss: 0.047
[44] loss: 0.046
[45] loss: 0.045
[46] loss: 0.044
[47] loss: 0.043
[48] loss: 0.043
[49] loss: 0.042
[50] loss: 0.042


In [13]:
 #test the model
def modelTest1(Loader):
    lossList1= []
    trueLable1= []
    for i,(feature1,lable1) in enumerate(Loader):
        inputs1= torch.tensor(feature1)
        output1= model1(inputs1)
        loss1= criterion1(output1,inputs1)
        trueLable1.append(lable1)
        lossList1.append(loss1.item())
        lossValue1= loss1.item()
        f= open("./modle1Loss.txt",'a')
        f.write(str(lossValue1) + '\n')
        '''
        if(i!=10):
            print("input:",inputs1)
            print("output:",output1)
            print("loss1:",loss1)
            print("lable:",lable1)
        else:
            assert(False)
        '''
    f.close()
    return lossList1,trueLable1
        

In [14]:
#test the model
lossList1,trueLable1= modelTest1(nonFraudulentTestLoader)
len(lossList1)
lossList1[1]

0.04311539605259895

In [15]:
len(trueLable1)
int(trueLable1[0])

0

In [16]:
#Network architecture for the base autoencoders
class autoencoder2(nn.Module):
    def __init__(self):
        super(autoencoder2,self).__init__()
        self.encoder2= nn.Sequential(
            nn.Linear(97,87),
            nn.LeakyReLU(True),
            nn.Linear(87,77),
            nn.LeakyReLU(True),
            nn.Linear(77,55),
            nn.LeakyReLU(True),
            nn.Linear(55,45),
            nn.LeakyReLU(True),
            nn.Linear(45,25),nn.LeakyReLU(True),nn.Linear(25,10))
       
        self.decoder2= nn.Sequential(
            nn.Linear(10,25),
            nn.LeakyReLU(True),
            nn.Linear(25,45),
            nn.LeakyReLU(True),
            nn.Linear(45,55),
            nn.LeakyReLU(True),
            nn.Linear(55,77),
            nn.LeakyReLU(True),
            nn.Linear(77,87),nn.LeakyReLU(True),nn.Linear(87,97),nn.Tanh())
    
    def forward(self,x):
        x= self.encoder2(x)
        x= self.decoder2(x)
        return x;

In [17]:
model2= autoencoder2()
criterion2= nn.MSELoss()
optimizer2= torch.optim.SGD(model2.parameters(), lr=0.001,momentum=0.9)

In [18]:
#train the model2
for epoch in range(300):
    running_loss2= 0.0
    for i, (feature2,lable2) in enumerate(fraudulentTestLoader):
        
        #gets the inputs
        inputs2= torch.tensor(feature2)
        lables2= torch.tensor(lable2)
        lables2= lables2.type(torch.LongTensor)
       
        # =====================forward====================
        output2= model2(inputs2)
        loss2= criterion2(output2,inputs2)
        
        # ===================backward====================
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
        
        # =======print the statistics
        running_loss2 += loss2.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss2 / fraudulentCarfeatures.shape[0]))
      #running_loss = 0.0

[1] loss: 0.048
[2] loss: 0.042
[3] loss: 0.036
[4] loss: 0.031
[5] loss: 0.026
[6] loss: 0.023
[7] loss: 0.020
[8] loss: 0.019
[9] loss: 0.018
[10] loss: 0.018
[11] loss: 0.017
[12] loss: 0.017
[13] loss: 0.017
[14] loss: 0.017
[15] loss: 0.017
[16] loss: 0.017
[17] loss: 0.017
[18] loss: 0.017
[19] loss: 0.017
[20] loss: 0.017
[21] loss: 0.017
[22] loss: 0.017
[23] loss: 0.017
[24] loss: 0.017
[25] loss: 0.017
[26] loss: 0.017
[27] loss: 0.017
[28] loss: 0.017
[29] loss: 0.017
[30] loss: 0.017
[31] loss: 0.017
[32] loss: 0.017
[33] loss: 0.017
[34] loss: 0.017
[35] loss: 0.017
[36] loss: 0.017
[37] loss: 0.017
[38] loss: 0.017
[39] loss: 0.017
[40] loss: 0.017
[41] loss: 0.017
[42] loss: 0.017
[43] loss: 0.017
[44] loss: 0.017
[45] loss: 0.017
[46] loss: 0.017
[47] loss: 0.017
[48] loss: 0.017
[49] loss: 0.017
[50] loss: 0.017
[51] loss: 0.017
[52] loss: 0.017
[53] loss: 0.017
[54] loss: 0.017
[55] loss: 0.017
[56] loss: 0.017
[57] loss: 0.017
[58] loss: 0.017
[59] loss: 0.017
[60] l

In [19]:
def modelTest2(Loader2):
    lossList2= []
    trueLable1= []
    for i,(feature2,lable2) in enumerate(Loader2):
        inputs2= torch.tensor(feature2)
        output2= model2(inputs2)
        loss2= criterion2(output2,inputs2)
        trueLable1.append(lable2)
        lossList2.append(loss2.item()) #put the loss
        '''        
        if(i!=10):
            print("input:",inputs2)
            print("output:",output2)
            print("loss1:",loss2)
            print("lable:",lable2)
        else:
            assert(False)
        '''
        lossValue2= str(loss2.item())
        f= open("./modle2Loss.txt",'a')
        f.write(lossValue2+'\n')
    f.close()
        
    return lossList2,trueLable1
        

In [20]:
lossList2,trueLable1= modelTest2(nonFraudulentTestLoader)


In [21]:
print("loss1:",lossList1[1])
print("loss2:",lossList2[1])
len(trueLable1)

loss1: 0.04311539605259895
loss2: 0.07364572584629059


3624

In [22]:
predictedLable1=[]
for k in range(len(lossList1)):
    if(lossList1[k]<lossList2[k]):
        predictedLable1.append(0)
    else:
        predictedLable1.append(1)

In [23]:

class1= predictedLable1.count(1)
class0= predictedLable1.count(0)
print('%d %d' %(class1,class0))
len(trueLable1)


294 3330


3624

In [24]:
#test model1 with another type of data
lossList3,trueLable2= modelTest1(fraudulentTestLoader)


In [25]:
#test model2 with another type of data
lossList4,trueLable2= modelTest2(fraudulentTestLoader)


In [26]:
predictedLable2=[]
for k in range(len(lossList3)):
    if(lossList3[k]<lossList4[k]):
        predictedLable2.append(0)
    else:
        predictedLable2.append(1)

In [27]:
class1= predictedLable2.count(1)
class0= predictedLable2.count(0)
print('%d %d' %(class1,class0))


40 191


In [28]:
totalLength1= len(trueLable1)
TP=FP=FN=TN= 0
for i in range(totalLength1):
    if(int(trueLable1[i])==1 and predictedLable1[i]==1):
        TP += 1
    elif(int(trueLable1[i])==1 and predictedLable1[i]==0):
        FN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==0):
        TN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==1):
        FP += 1

totalLength2= len(trueLable2)
for i in range(totalLength2):
    if(int(trueLable2[i])==1 and predictedLable2[i]==1):
        TP += 1
    elif(int(trueLable2[i])==1 and predictedLable2[i]==0):
        FN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==0):
        TN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==1):
        FP += 1
        
print("\nResult ")
print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)
print("\n")
print("\nTP:",TP)
print("\nFN:",FN)
print("\nFP:",FP)
print("\nTN:",TN)



Result 
Accuracy: 87.4189364461738
Sensitivity: 17.316017316017316
Specificity: 91.88741721854305



TP: 40

FN: 191

FP: 294

TN: 3330
